In [11]:
import pandas as pd
import random
import re
import json

random.seed(2)

which_wiki = 1

# Read in the data
df = pd.read_csv(f'all-outputs-new-wiki-{which_wiki}.csv')

In [12]:
annotater_dict = json.load(open('annotater_dict-new-wiki-1.json'))
id_dict = json.load(open('id_dict-new-wiki-1.json'))

# print length of each annotater's list
for annotater in annotater_dict:
    print(annotater, len(annotater_dict[annotater]))

vinayak 15
kelly 15
rachel 15
vishnesh 15
ayush 15
anton 15


In [15]:
split_idx = 50

# Human 1 Writing,Human 1,Human 2 Writing,Human 2,GPT-3-zero-shot,GPT-3-few-shot,Muss,T5-3B,T5-11B
system_list = ["Human 1 Writing", "Human 2 Writing", "GPT-3-zero-shot", "GPT-3-few-shot", "Muss", "T5-3B", "T5-11B"]
for annotater in annotater_dict:
    annotater_batch = []
    start_id = 0
    for id in annotater_dict[annotater]:
        # get the id th row of df
        row = df.iloc[id]
        random.seed(2)
        # random shuffle the system list
        random.shuffle(system_list)
        for system in system_list:
            current_dict = {}
            current_dict["id"] = start_id
            current_dict["original"] = row["Original Sentence"].strip()
            current_dict["original_spans"] = []

            simplified = row[system].strip()
            simplified = re.sub(r" 's", "'s", simplified)
            current_dict["simplified"] = simplified
            current_dict["simplified_spans"] = []

            if "||" in simplified:
                #  find index of all || in simplified
                indices = [m.start() for m in re.finditer('\|\|', simplified)]
                for i, indice in enumerate(indices):
                    current_dict["simplified_spans"].append([2, indice, indice+2, i])
            
            current_dict["system"] = f"new-wiki-{which_wiki}/{system}"
            annotater_batch += [current_dict]
            start_id += 1

    # split annotater_batch into 2 batches first 40 and last 35
    annotater_batch_1 = annotater_batch[:split_idx]
    annotater_batch_2 = annotater_batch[split_idx:]

    with open(f"../batches/new-wiki-{which_wiki}/part1/{annotater}.json", "w") as f:
        json.dump(annotater_batch_1, f, indent=4)

    # reindex annotater_batch_2's id from 0
    for i, item in enumerate(annotater_batch_2):
        item["id"] = i

    with open(f"../batches/new-wiki-{which_wiki}/part2/{annotater}.json", "w") as f:
        json.dump(annotater_batch_2, f, indent=4)